In [1]:
"""
 author: P. Reyes
 date: Nov 4, 2022
 description: This code allows to do queries to the django-based SRI AMISR database
              just reading the sqlite3 database file. This code uses sqlalchemy and
              is independent of django.
"""
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
# dbPath = '/opt/websites/database/calendar/isr_database/amisrdb'
dbPath = 'amisrdb'
Base = automap_base()
engine = sqlalchemy.create_engine('sqlite:///%s'%dbPath)
Base.prepare(engine, reflect=True)
session = sqlalchemy.orm.Session(engine)
db = Base.classes

In [2]:
q_insts = session.query(db.procdb_instrument)
for inst0 in q_insts:
    print(f"instrument abbr:{inst0.abbr}, instrument id = {inst0.id}")

instrument abbr:PFISR, instrument id = 1
instrument abbr:RISR-N, instrument id = 2


In [3]:
# PFISR experiments
q_pfisr = session.query(db.procdb_experiment).filter(db.procdb_experiment.inst_id == 1)
q_risrn = session.query(db.procdb_experiment).filter(db.procdb_experiment.inst_id == 2)
print(f"{q_pfisr.count()} PFISR experiments. {q_risrn.count()} RISR-N experiments")

20811 PFISR experiments. 7762 RISR-N experiments


In [4]:
merged_exps_pfisr = {}
for exp0 in q_pfisr:
    if not exp0.master_exp:
        # if no primary experiment then skip
        continue
    if exp0.master_exp != exp0.name:
        if exp0.master_exp not in merged_exps_pfisr.keys():
            merged_exps_pfisr.update({exp0.master_exp:[]})
        merged_exps_pfisr[exp0.master_exp].append(exp0.name)

In [5]:
with open('merged_pfisr_exps.txt','w') as fp:
    for item,val in merged_exps_pfisr.items():
        fp.write("{}:".format(item))
        fp.write(",".join(val))
        fp.write("\n")

In [6]:
merged_exps_risrn = {}
for exp0 in q_risrn:
    if not exp0.master_exp:
        # if no primary experiment then skip
        continue
    if exp0.master_exp != exp0.name:
        if exp0.master_exp not in merged_exps_risrn.keys():
            merged_exps_risrn.update({exp0.master_exp:[]})
        merged_exps_risrn[exp0.master_exp].append(exp0.name)

In [7]:
with open('merged_risrn_exps.txt','w') as fp:
    for item,val in merged_exps_risrn.items():
        fp.write("{}:".format(item))
        fp.write(",".join(val))
        fp.write("\n")

# New Stuff

In [18]:
import sqlalchemy as db
import datetime as dt

In [50]:
unixstarttime = dt.datetime(2014,2,12,0,0,0)
unixendtime = dt.datetime(2014,2,13,0,0,0)

engine = db.create_engine('sqlite:///amisrdb')
insp = db.inspect(engine)
print(insp.get_table_names())
with engine.connect() as conn:
    
    metadata = db.MetaData()
    print(metadata.tables.keys())
    
    exp = db.Table('procdb_experiment', db.MetaData(), autoload=True, autoload_with=engine)
    print(exp.columns.keys())
    
    # query = db.select([exp])
    # print(conn.execute(query).fetchall())

# In procdb_insrument:
# ['id', 'abbr', 'descriptor', 'latitude', 'longitude', 'code']
# [(1, 'PFISR', 'Poker Flat Incoherent Scatter Radar', 0.0, 0.0, 61), (2, 'RISR-N', 'Resolute Bay Incoherent Scatter Radar - North', 0.0, 0.0, 91)]
# 1 = PFISR
# 2 = RISR-N

# procdb_experimenttype contains mode information
# label = mode name
# id??


    params = [exp.columns.name, exp.columns.master_exp, exp.columns.type_id, exp.columns.start_time, exp.columns.end_time]


    # queary AMISR database for experiments in this time frame
    condition = db.and_(exp.columns.inst_id==1, exp.columns.end_time>unixstarttime, exp.columns.start_time<unixendtime)
    # condition = db.and_(exp.columns.inst_id==1)
    # db.procdb_experiment.inst_id == 1
    query = db.select(params).where(condition)
    exp_list = conn.execute(query).fetchall()

    for exp in exp_list:
        print(exp.name, exp.master_exp, exp.start_time, exp.end_time, exp.type_id)

        exptype = db.Table('procdb_experimenttype', db.MetaData(), autoload=True, autoload_with=engine)
        # print(exp.columns.keys())
        query = db.select([exptype.columns.label]).where(exptype.columns.id == exp.type_id)
        expname = conn.execute(query).fetchall()
        print(expname)


['auth_group', 'auth_group_permissions', 'auth_message', 'auth_permission', 'auth_user', 'auth_user_groups', 'auth_user_user_permissions', 'django_admin_log', 'django_content_type', 'django_evolution', 'django_project_version', 'django_session', 'django_site', 'procdb_experiment', 'procdb_experiment_tags', 'procdb_experimenttag', 'procdb_experimenttype', 'procdb_instrument', 'procdb_plannedexperiment', 'procdb_processingstatus']
dict_keys([])
['status_id', 'name', 'non_spinning_backup', 'start_time', 'id', 'note', 'proc_log', 'inst_id', 'end_time', 'type_id', 'cloud_backup', 'items', 'master_exp', 'status_date', 'size']
20140212.001  2014-02-12 04:55:53 2014-02-12 05:01:52 605
[('Swarm_v01_20140212.005',)]
20140212.003  2014-02-12 09:03:49 2014-02-12 09:06:00 606
[('CalSphere08_27392_20140212.075',)]
20140212.005  2014-02-12 12:07:49 2014-02-12 12:11:01 607
[('CalSphere08_22824_20140212.085',)]
20140212.007  2014-02-12 17:19:52 2014-02-12 17:25:51 608
[('Swarm_v01_20140212.006',)]
2014

To form experiment path, need:
- start date (year, month)
- mode name
- experiment number
- master experiment name/number, if relevant

In [76]:
starttime = dt.datetime(2014,2,12,0,0,0)
endtime = dt.datetime(2014,2,13,0,0,0)

import sqlalchemy
from sqlalchemy.ext.automap import automap_base

# Set up DB
# dbPath = '/opt/websites/database/calendar/isr_database/amisrdb'
dbPath = 'amisrdb'
Base = automap_base()
engine = sqlalchemy.create_engine('sqlite:///%s'%dbPath)
Base.prepare(engine, reflect=True)
session = sqlalchemy.orm.Session(engine)
db = Base.classes

# Get instrument ID number
inst_id = session.query(db.procdb_instrument).filter(db.procdb_instrument.abbr == 'PFISR').one().id
print(inst_id)

# Find experiment by start/end times and radar id
filt = sqlalchemy.and_(db.procdb_experiment.inst_id == inst_id, db.procdb_experiment.end_time>starttime, db.procdb_experiment.start_time<endtime)
q_pfisr = session.query(db.procdb_experiment).filter(filt)
for q in q_pfisr:
    # print(q.name, q.master_exp, q.type_id)
    
    # Get Mode
    mode = session.query(db.procdb_experimenttype).filter(db.procdb_experimenttype.id == q.type_id).one().label
    # print(mode)
    
    if q.master_exp:
        start_date = dt.datetime.strptime(q.master_exp[0:8], '%Y%m%d')
    else:
        start_date = dt.datetime.strptime(q.name[0:8], '%Y%m%d')
    
    print('Start Date:', start_date)
    print('Mode:', mode)
    print('Experiment:', q.name)
    print('Mast. Experiment:', q.master_exp)
    


1
Start Date: 2014-02-12 00:00:00
Mode: Swarm_v01_20140212.005
Experiment: 20140212.001
Mast. Experiment: 
Start Date: 2014-02-12 00:00:00
Mode: CalSphere08_27392_20140212.075
Experiment: 20140212.003
Mast. Experiment: 
Start Date: 2014-02-12 00:00:00
Mode: CalSphere08_22824_20140212.085
Experiment: 20140212.005
Mast. Experiment: 
Start Date: 2014-02-12 00:00:00
Mode: Swarm_v01_20140212.006
Experiment: 20140212.007
Mast. Experiment: 
Start Date: 2014-02-12 00:00:00
Mode: CalSphere08_5398_20140212.112
Experiment: 20140212.009
Mast. Experiment: 
Start Date: 2014-02-12 00:00:00
Mode: CalSphere08_22824_20140212.091
Experiment: 20140212.011
Mast. Experiment: 
Start Date: 2014-02-12 00:00:00
Mode: CalSphere08_25398_20140212.091
Experiment: 20140212.013
Mast. Experiment: 
Start Date: 2014-02-09 00:00:00
Mode: IPY27
Experiment: 20140211.014
Mast. Experiment: 20140209.007
Start Date: 2014-02-09 00:00:00
Mode: IPY27
Experiment: 20140212.002
Mast. Experiment: 20140209.007
Start Date: 2014-02-09 0